In [1]:
import polars as pl
from statsmodels.formula import api as smf

Read datasets as downloaded from [kaggle](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020).

In [2]:
results = pl.read_csv("data/raw/results.csv", ignore_errors=True)
status = pl.read_csv("data/raw/status.csv")
drivers = pl.read_csv("data/raw/drivers.csv")
races = pl.read_csv("data/raw/races.csv")
constructors = pl.read_csv("data/raw/constructors.csv")

Since our model will try to predict the finishing position, we introduce a dummy variable to account for retirements from the race that the driver had no fault in. 
These are mostly techinical failures.
There are some categories which have som abiguity even after checking the races where this occured on wikipedia.

In [3]:
not_self_responsible_dnf = [
    "Disqualified", # mixed bag, includes driver erros but many team errors
    "Ignition",
    "Engine",
    "Gearbox",
    "Transmission",
    "Clutch",
    "Hydraulics",
    "Electrical",
    "Radiator",
    "Suspension",
    "Brakes",
    "Differential",
    "Overheating",
    "Mechanical",
    "Tyre",
    "Driver Seat",
    "Puncture",
    "Driveshaft",
    "Retired", # this is a bit of a mixed bag - some due to technical, some after collisions
    "Fuel pressure",
    "Front wing",
    "Water pressure",
    "Refuelling",
    "Wheel",
    "Throttle",
    "Steering",
    "Technical",
    "Electronics",
    "Broken wing",
    "Heat shield fire",
    "Exhaust",
    "Oil leak",
    "Wheel rim",
    "Water leak",
    "Fuel pump",
    "Track rod",
    "Oil pressure",
    "Engine fire",
    "Engine misfire",
    "Tyre puncture",
    "Out of fuel",
    "Wheel nut",
    "Pneumatics",
    "Handling", # found nothing on wikipedia indicating driver error
    "Rear wing",
    "Fire",
    "Wheel bearing",
    "Physical", # found nothing on wikipedia indicating driver error
    "Fuel system",
    "Oil line",
    "Fuel rig",
    "Launch control",
    "Fuel",
    "Power loss",
    "Vibrations",
    "Safety",
    "Drivetrain",
    "Ignition"
    "Chassis",
    "Battery",
    "Halfshaft",
    "Crankshaft",
    "Safety concerns", # found nothing on wikipedia indicating driver error
    "Alternator",
    "Underweight",
    "Safety belt",
    "Oil pump",
    "Fuel leak",
    "Excluded", # usually team failing to follow regulations
    "Injection",
    "Distributor",
    "Driver unwell", # not related to skill
    "Turbo",
    "CV joint",
    "Water pump",
    "Spark plugs",
    "Fuel pipe",
    "Oil pipe",
    "Axle",
    "Water pipe",
    "Magneto",
    "Supercharger",
    "Power Unit",
    "ERS",
    "Brake duct",
    "Seat",
    #"Debris", # is hitting debris the drivers fault?
    "Illness", # nothing to do with driver skill
    "Undertray",
    "Cooling system",
    # let's not blame the victims
    "Injury",
    "Fatal accident",
    "Eye injury", # Helmut Marko for no fault of his own 
    "Injured", # relates to injuries sustained before the race
    "Injury", # relates to injuries sustained before the race
]

These DNF reasons are treated as being the drivers fault:

In [4]:
status.filter((~pl.col("status").is_in(not_self_responsible_dnf)) & (~pl.col("status").str.contains("Lap")))["status"].unique().to_list()

['Withdrew',
 'Damage',
 'Finished',
 'Accident',
 '107% Rule',
 'Stalled',
 'Did not qualify',
 'Not restarted',
 'Did not prequalify',
 'Not classified',
 'Debris',
 'Collision',
 'Chassis',
 'Collision damage',
 'Spun off']

In [5]:
finishing_position = "positionOrder"
df = (
    results
    # remove drivers with multiple entries per race and use their best
    .sort(["raceId", "driverId", finishing_position])
    .unique(["raceId", "driverId"], keep="first")
    # build feature dnf
    .join(status, on="statusId", how="left")
    .with_columns(
        pl.col("status").is_in(not_self_responsible_dnf).alias("technical_dnf"),
    )
    # build target
    .with_columns(pl.col(finishing_position).max().over("raceId").alias(f"{finishing_position}_last"))
    .with_columns(
        score = 1 - ((pl.col(finishing_position) - 1) / (pl.col(f"{finishing_position}_last") - 1)),
    )
    # join more data
    .join(drivers, on="driverId", how="left")
    .join(races, on="raceId", how="left")
    # remove indianapolis 500
    .filter(~pl.col("name").str.contains("Indianapolis 500"))
    # remove drivers with not enough experience
    .filter(pl.col("raceId").n_unique().over("driverId") > 20)
    # build more human readable names
    .with_columns(
        race=pl.col("year").cast(pl.Utf8) + " " + pl.col("name"),
        driver=pl.col("forename") + " " + pl.col("surname") + pl.when(pl.col("code") != r"\N").then(" ("+  pl.col("code") + ")").otherwise(pl.lit("")),
    )
    # split some driver careers in half
    # this is inspired by https://doi.org/10.1515/jqas-2015-0050
    # here they notice an unexpectedly high ranking of Nico Rosberg 
    # this can be mitigated by treating MSC carrer as two seperate drivers (one extra for post retirement)
    # A similiar effect can be seen with Norris, but in a more extreme case (he is promoted to top3 of all time):
    # He comfortably outperfomred Daniel Ricciardo at Mclaren while Ricciardo could easily keep up with Max Verstappen at RedBull 
    # He also performed similiar to Sainz at McLaren, who in tern is about on a Level with Leclerc at Ferrari.
    # Leclerc also outperformed Sebastian Vettel at his last year at Ferrari.
    # If you assume same driver performance across all seasons this gives Norris a massive boost.
    # But for experts of the sport it was clear that both Vettel and Ricciardo hit a personal low in those years.
    # This can be accounted for with the code below.
    .with_columns(
        driver=pl.when(pl.col("driver").str.contains("Michael Schumacher") & (pl.col("year") > 2006))
        .then(pl.col("driver") + " post-retirement")
        .otherwise(pl.col("driver"))
    )
    .with_columns(
        driver=pl.when(pl.col("driver").str.contains("Sebastian Vettel") & (pl.col("year") > 2019))
        .then(pl.col("driver") + " slump")
        .otherwise(pl.col("driver"))
    )
    .with_columns(
        driver=pl.when(pl.col("driver").str.contains("Daniel Ricciardo") & (pl.col("year") > 2020))
        .then(pl.col("driver") + " slump")
        .otherwise(pl.col("driver"))
    )
    # build car info
    .join(constructors, on="constructorId", how="left", suffix="_constructor")
    # define car era based on major changes in the cars due to rule changes or techincal development
    # many times in changes of an era, the hierarchy of teams got mixed up since teams lost their techincal advantages
    # eras taken from: https://en.wikipedia.org/wiki/History_of_Formula_One
    .with_columns(
        pl.when(pl.col("year") < 1958)
        .then(pl.lit("Front-Engines"))
        .when(pl.col("year") < 1962)
        .then(pl.lit("Mid-Engines"))
        .when(pl.col("year") < 1968)
        .then(pl.lit("1.5-Litres"))
        .when(pl.col("year") < 1977)
        .then(pl.lit("V12s"))
        .when(pl.col("year") < 1983)
        .then(pl.lit("Ground-Effect"))
        .when(pl.col("year") < 1989)
        .then(pl.lit("Turbos"))
        .when(pl.col("year") < 1994)
        .then(pl.lit("Traction-Control"))
        .when(pl.col("year") < 2000)
        .then(pl.lit("Improved-Safety"))
        .when(pl.col("year") < 2006)
        .then(pl.lit("V10s"))
        .when(pl.col("year") < 2009)
        .then(pl.lit("V8s"))
        .when(pl.col("year") < 2014)
        .then(pl.lit("KERS"))
        .when(pl.col("year") < 2022)
        .then(pl.lit("V6-Turbo-Hybrids"))
        .otherwise(pl.lit("Ground-Effect 2"))
        .alias("era")
    )
    .with_columns(
        #car=pl.col("name_constructor") + " ("+  pl.col("era") + ")",
        car=pl.col("year").cast(pl.Utf8)  +" "+ pl.col("name_constructor"), # looking at cars by year instead of era perfomred better
    )
    # remove drivers who drive too few different cars
    .with_columns(pl.col("year").n_unique().over("driverId").alias("n_seasons_driver"))
    .filter(pl.col("n_seasons_driver") > 1)
    # remove cars used too rarely - eg indianapolis 500 runners
    .with_columns(pl.col("raceId").n_unique().over("car").alias("n_races_car"))
    .filter(pl.col("n_races_car") > 1)
    # reduce columns
    .select(["raceId", "driverId", "race", "driver", "car", "technical_dnf", "score"])
).to_pandas().set_index(["raceId", "driverId", "race"])

In [6]:
df.loc[1].sort_values("score", ascending=False)

,,driver,car,technical_dnf,score
driverId,race,,,,
18,2009 Australian Grand Prix,Jenson Button (BUT),2009 Brawn,False,1.000000
22,2009 Australian Grand Prix,Rubens Barrichello (BAR),2009 Brawn,False,0.947368
15,2009 Australian Grand Prix,Jarno Trulli (TRU),2009 Toyota,False,0.894737
10,2009 Australian Grand Prix,Timo Glock (GLO),2009 Toyota,False,0.842105
4,2009 Australian Grand Prix,Fernando Alonso (ALO),2009 Renault,False,0.789474
3,2009 Australian Grand Prix,Nico Rosberg (ROS),2009 Williams,False,0.736842
67,2009 Australian Grand Prix,Sébastien Buemi (BUE),2009 Toro Rosso,False,0.684211
7,2009 Australian Grand Prix,Sébastien Bourdais (BOU),2009 Toro Rosso,False,0.631579
16,2009 Australian Grand Prix,Adrian Sutil (SUT),2009 Force India,False,0.578947


Here is an example of the dataset used for modeling.
Most interesting is the `score` variable, which describes a drivers finishing position.
A value of `score=0` means the driver finished last.
A value of `score=1` means the driver won the race.
The model will try to preidct this `score`.
The idea is that the finishing position is a good dependent variable when evalutating talent - if analysed together with other mitigating factors.
Most importantly those include the `car` and eventual techinical failures of the car `techincal_dnf`.
This is inspired by the method established in [this paper](https://www.researchgate.net/publication/228310743)'s section 3.

By furthermore restricting the `score` to `[0, 1]` we account for the fact that finishing 5th in a race of 10 competitors is a different achievement than finishing 5th in a race of 20.

In [7]:
model = smf.logit("score ~ driver + car + technical_dnf - 1", df).fit()

Optimization terminated successfully.
         Current function value: 0.546437
         Iterations 9


The models parameters will be our indicator to figure out the greatest of all time - both drivers and cars.

One obvious weakness of the model is that it assumes a somewhat constant driver performance over all years.
This can become a problem for some cases, which are discussed in the source code comments above.

Since a modern grid contains 20 drivers, we will show the top 20 for an all-star grid.
For the cars, we are only concerned with a top 10.

In [8]:
res = model.summary2().tables[1]
drivers = res.index.str.startswith("driver")
cars = res.index.str.startswith("car")

In [9]:
res.loc[drivers].sort_values("Coef.", ascending=False).head(20)

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
driver[Juan Fangio],2.017850,0.783447,2.575605,0.010006,0.482322,3.553377
driver[Jim Clark],1.867607,0.950665,1.964527,0.049469,0.004338,3.730876
driver[Jackie Stewart],1.742802,0.960607,1.814271,0.069636,-0.139954,3.625558
driver[Max Verstappen (VER)],1.698323,1.032919,1.644198,0.100135,-0.326160,3.722807
driver[Emerson Fittipaldi],1.684459,0.968659,1.738959,0.082042,-0.214078,3.582996
driver[Michael Schumacher (MSC)],1.630182,0.983229,1.657989,0.097320,-0.296910,3.557275
driver[Mike Hawthorn],1.587892,0.924388,1.717777,0.085837,-0.223875,3.399659
driver[Dan Gurney],1.578747,0.930396,1.696855,0.089724,-0.244796,3.402289
driver[Sebastian Vettel (VET)],1.524783,1.008826,1.511444,0.130675,-0.452478,3.502045
driver[Fernando Alonso (ALO)],1.522687,0.990488,1.537310,0.124218,-0.418634,3.464008


Even though Lewis Hamilton is by number of victories and title clearly the most successful driver of all time, he ranks fairly low in this analysis.
This can be explained by looking at the cars scores.
Many of the cars with the strongest rating were driven by Hamilton in his dominant years.
This highlights another difficulty when evaluating driver performance:
A driver can only ever be truly judged against his team mate.
But if the team mate is perfroming on a high level as well it is difficult to judge of the results are mainly determined by the drivers skills or the cars performance.
Hamilton had strong team mates for most of his career including 3 drivers champions (Fernando Alonso, Jenson Button, Nico Rosberg).
Another prominent example are Ayrton Senna and Alain Prost who competed alongside each other in McLaren.

In [10]:
res.loc[cars].sort_values("Coef.", ascending=False).head(10)

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
car[T.1954 Mercedes],1.391580,2.448258,0.568396,0.569766,-3.406916,6.190077
car[T.1988 McLaren],1.199490,1.144319,1.048213,0.294541,-1.043335,3.442315
car[T.2017 Mercedes],1.065616,1.119895,0.951532,0.341334,-1.129338,3.260570
car[T.2019 Mercedes],1.029466,1.111759,0.925980,0.354457,-1.149541,3.208474
car[T.2023 Red Bull],0.938729,1.233033,0.761317,0.446468,-1.477971,3.355430
car[T.2015 Mercedes],0.879950,1.131218,0.777878,0.436641,-1.337196,3.097095
car[T.2011 Red Bull],0.856814,1.129276,0.758729,0.448015,-1.356525,3.070153
car[T.1991 McLaren],0.832957,1.071856,0.777117,0.437090,-1.267842,2.933757
car[T.2018 Mercedes],0.832680,1.093337,0.761595,0.446302,-1.310222,2.975581
car[T.2020 Mercedes],0.809329,1.116243,0.725047,0.468423,-1.378467,2.997125


Some reality checks for the dummy variables.

In [11]:
res.loc[~drivers & ~cars]

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
technical_dnf[T.True],-1.355607,0.037035,-36.603118,2.551345e-293,-1.428195,-1.283019


In [12]:
drivername = "Schumacher"
res.loc[res.index.str.contains(drivername)]

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
driver[Michael Schumacher (MSC)],1.630182,0.983229,1.657989,0.097320,-0.296910,3.557275
driver[Michael Schumacher (MSC) post-retirement],0.776760,1.112713,0.698078,0.485129,-1.404117,2.957637
driver[Mick Schumacher (MSC)],0.538362,1.226308,0.439010,0.660654,-1.865159,2.941882
driver[Ralf Schumacher (SCH)],0.805959,0.998456,0.807205,0.419549,-1.150980,2.762897


In [30]:
from plotly import express as px
import plotly.graph_objects as go

goats = list(res.loc[drivers].sort_values("Coef.", ascending=False).head(20).index.str.replace("driver[", "").str.replace("]", ""))
goats = df[df["driver"].isin(goats)]

fig1 = px.scatter(
    y=goats["score"],
    x=model.predict(goats),
    color=goats["driver"],
    #symbol=goats["car"],
    hover_name=goats.reset_index()["race"],
)
fig2 = px.line((0,1), (0,1),color_discrete_sequence=["black"])
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.update_xaxes(title_text="Prediction")
fig3.update_yaxes(title_text="Race Result")
fig3.show()

The graph above compares the models predictions with observed race results.
The black diagonal marks where predictions and results exactl match.
All result above indicate a driver performing above expectations.
All results below indicate a driver performing below expectations.

This visualization also allows to see the most dominant driver + car pairings over the years.
The more to the right, the higher a combination scores.

Undisputed leader is Juan Manuel Fangio in his 1954 Mercedes.
Quite a bit off in second is Max Verstappen in 2023 (this analysis was done mid season 2023, so probably subject to change).
A close third is Alain Prost in his 1988 McLaren.
This is followed by Lewis Hamilton in his 2017 and 2019 Mercedes.